In [1]:

!pip install wget

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=49bad69b94ba7f9ea14df1d972009d4976ce06268a2178463452736bc7dc6e6d
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\40\b3\0f\a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [8]:
import wget
import os


url = 'https://download.pytorch.org/tutorial/hymenoptera_data.zip'

os.makedirs("data", exist_ok = True)
#파일다운로드
zip_path = wget.download(url, out = "data/hymenoptera_data.zip")
#파라미터 url, out="받은파일경로"
#다운받은 파일의 경로를 반환

#다운받은 zip파일 압축 풀기
#zip 압축 다루는 표준 모듈 : zipfile
from zipfile import ZipFile
#압축 파일연결 -> 압축 풀기 -> 연결 닫기

with ZipFile(zip_path) as zf: #풀 압축파일의 경로를 넣어서 객체 생성
    zf.extractall("data") #압축을 풀 경로, 생략시 현재디렉토리


In [18]:

from glob import glob # 파일 * 디렉토리 **

p = glob("data/**/*.jpg",recursive=True)
## data 아래에 있는 디렉토리
glob("data/**/",recursive=True)

##복사 ->shutil 표준모듈. 
#copy(src,target) : 파일 + 권한정보
#copy2(src,target) : 파일 + 파일의 메타정보(생성일자, 수정일자,...)
#copyfile(src,target) : 파일만
#copytree(src,target) :디렉토리 복사
import shutil
#shutil.copyfile(p, "data/")
# 이미지 파일들이 있는 디렉토리 조회
img_dir_path = glob("data/**/")[0] #카피 후 디렉토리삭제를 위해 경로 저장
file_list = glob("data/**/*.jpg", recursive=True)

#복사
for file in file_list:
    shutil.copy2(file, "data")


In [20]:

#불필요한 원본과 경로 삭제, 다운받은 압축파일 삭제
try:
    os.remove(zip_path)
except:
    print("삭제할 파일이 없음")

try:
    shutil.rmtree(img_dir_path)
except:
    print("삭제할 디렉토리 없음")


삭제할 파일이 없음
삭제할 디렉토리 없음


# 이미지 전처리
- 이진화 
- 좌우 이동
- 밝기 변경
- 엣지 디텍션
- 모든 이미지 위 4가지 처리를 한 파일을 각각 생성.

In [24]:
#파일 경로의 정보 조회
fname = 'data\\1095476100_3906d8afde.jpg'
print(os.path.basename(fname))
print(os.path.dirname(fname))
print(os.path.splitext('1095476100_3906d8afde.jpg')) #파일명과 확장자 분리


1095476100_3906d8afde.jpg
data
('1095476100_3906d8afde', '.jpg')


['data\\0013035.jpg',
 'data\\1030023514_aad5c608f9.jpg',
 'data\\10308379_1b6c72e180.jpg',
 'data\\1032546534_06907fe3b3.jpg',
 'data\\1053149811_f62a3410d3.jpg',
 'data\\1073564163_225a64f170.jpg',
 'data\\10870992_eebeeb3a12.jpg',
 'data\\1092977343_cb42b38d62.jpg',
 'data\\1093831624_fb5fbe2308.jpg',
 'data\\1095476100_3906d8afde.jpg']

In [25]:
import cv2
import numpy as np

img_list = glob("data/*.jpg")
cnt = 0

for img_path in img_list:
    fname = os.path.basename(img_path) #파일명 조회
    fname, ext = os.path.splitext(fname) #파일 이름, 확장자를 분리해서 저장
    #읽기
    img = cv2.imread(img_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #이미지 이진화
    _, img_bin = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
    save_path1= f"data/{fname}_binary_{cnt:03d}{ext}" #0으로 세자리를 맞추는 정수
    cv2.imwrite(save_path1, img_bin)

    ##좌우 이동 20픽샐
    affine_mat = np.array([[1,0,20],[0,1,0]], dtype="float32")
    img_move = cv2.warpAffine(img, affine_mat,(0,0))
    save_path2 = f"data/{fname}_move_{cnt:03d}{ext}"
    cv2.imwrite(save_path2, img_move)

    ##밝기조절
    img_brighter = cv2.add(img,30)
    save_path3 = f"data/{fname}_bright_{cnt:03d}{ext}"
    cv2.imwrite(save_path3, img_brighter)

    ##엣지디텍션
    img_edge = cv2.Canny(img_gray, 50, 150)
    save_path4 = f"data/{fname}_edge_{cnt:03d}{ext}"
    cv2.imwrite(save_path4, img_edge)
